In [1]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
import pandas as pd
import re

In [58]:
# Load train dataset
ds = load_dataset("ailsntua/QEvasion")

# Convert to pandas and keep only useful columns
df_train = ds["train"].to_pandas()[["question","interview_answer",
                                   "label"]]

In [60]:
df_train.head(5)

,question,interview_answer,label
0,How would you respond to the accusation that t...,"\nThe President. Well, look, first of all, the...",Explicit
1,Do you think President Xi is being sincere abo...,"\nThe President. Well, look, first of all, the...",General
2,1. Q1: Do you believe the country's slowdown a...,"\nThe President. Look, I think China has a dif...",Partial/half-answer
3,2. Q2: Are you worried about the meeting betwe...,"\nThe President. Look, I think China has a dif...",Dodging
4,Is the President's engagement with Asian coun...,"\nThe President. Well, I hope I get to see Mr....",Explicit


In [61]:
# Regex explanation:

# ^ matches the start of the string
# (\d+\.|Part \d+:|Q\d*:|\d+\. Q\d*: ) is a capturing group that matches
# one of the following:
#     \d+\. : one or more digits followed by a period
#
#     Part \d+: : the string "Part " followed by one or more digits,
#     a colon, and an optional space
#
#     Q\d*: : the string "Q" followed by one or more digits, a colon, a
#     and an optional space
#
#     \d+\. Q\d*: : one or more digits followed by a period, a space,
#     "Q", one or more digits, a colon, and an optional space
#
#     - : start sentence with "-"

# Remove indexing from questions
index_pattern = r'^(\d+\. Q\d+:|\d+\.|Part \d+:|Q\d+:|-)'
df_train['question'] = df_train['question'].str.replace(index_pattern,
                                                        '', regex=True)

# Remove quotes and new line espace characters
df_train['question'] = df_train['question'].str.replace(r'["\n]',
                                                        '', regex=True)
df_train['interview_answer'] = df_train['interview_answer'].str.replace(
                                                            r'\n', '',
                                                            regex=True)

# Remove first sentence from answer
sentence_pattern = r'^[^.]+\.?'
df_train['interview_answer'] = df_train['interview_answer'].str.replace(
                                                        sentence_pattern,
                                                        '', regex=True)

# Remove description from questions

# List of exception indexes
exceptions = [142,493,699,809,1052,1053,1446,
              2417,2631,2821,3181,3390]

df_train.loc[~df_train.index.isin(exceptions), 'question'] = df_train.loc[
    ~df_train.index.isin(exceptions), 'question'].apply(
    lambda x: re.sub(r'^[^:]+: ', '', x))

In [62]:
df_train.head(5)

,question,interview_answer,label
0,How would you respond to the accusation that t...,"Well, look, first of all, the—I am sincere ab...",Explicit
1,Do you think President Xi is being sincere abo...,"Well, look, first of all, the—I am sincere ab...",General
2,Do you believe the country's slowdown and gro...,"Look, I think China has a difficult economic ...",Partial/half-answer
3,Are you worried about the meeting between Pre...,"Look, I think China has a difficult economic ...",Dodging
4,Is the President's engagement with Asian coun...,"Well, I hope I get to see Mr. Xi sooner than ...",Explicit


In [66]:
filtered_no_quest_df = df_train[~df_train['question'].str.contains('\?')]
len(filtered_no_quest_df)

772

In [67]:
len(df_train[df_train['question'].str.contains('\?')])

2676

In [68]:
filtered_no_quest_df

,question,interview_answer,label
8,Concerns about the lack of communication betw...,It's not a wedge issue of the Global South. I...,Explicit
9,Inquiry about the reaction of Kyiv regarding ...,It's not a wedge issue of the Global South. I...,Explicit
16,ensuring Finland that the U.S. will remain a r...,I absolutely guarantee it. There is no questi...,Explicit
18,Concerns about the comments motivating Putin ...,"First of all, no one can join NATO while the ...",Deflection
19,The risk of the war dragging on for years.,"First of all, no one can join NATO while the ...",Dodging
...,...,...,...
3403,I wonder what your reaction is to that,"You know, I really haven't—I'm still going to...",Explicit
3415,Asking for an explanation of not knowing somet...,"You didn't know it, either.",Dodging
3416,Adjustments to the agenda regarding Social Sec...,"I told—Ken, I told Hank Paulson to tell the M...",General
3430,Secretary Rumsfeld Accountability,"Peter, you're asking me why I believe Secreta...",Dodging
